<a href="https://colab.research.google.com/github/ernestoruizds/DS_C3_SC1_ERNESTO_RUIZ_SANCHEZ/blob/main/RetoEmpleados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
# 1. Importa las librerías requeridas
import pandas as pd

from google.colab import drive
from sklearn import linear_model
from datetime import datetime
from sklearn.decomposition import PCA

drive.mount('/content/drive', force_remount=True)

# 2. Lee el archivo CSV llamado empleadosRETO.csv y coloca los datos en un frame de Pandas llamado EmpleadosAttrition
EmpleadosAttrition = pd.read_csv("/content/drive/MyDrive/M15/Reto/empleadosRETO.csv")

# 3. Elimina las columnas que, con alta probabilidad, no tienen relación alguna con la salida.
#    Hay algunas columnas que contienen información que no ayuda a definir el desgaste de un empleado, tal es caso de las siguientes:
#    EmployeeCount: número de empleados, todos tienen un 1
#    EmployeeNumber: ID del empleado, el cual es único para cada empleado
#    Over18: mayores de edad, todos dicen “Y”
#    StandardHours: horas de trabajo, todos tienen “80”

EmpleadosAttrition = EmpleadosAttrition.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'],axis=1)

# 4. Analiza la información proporcionada, si detectaste que no se cuenta con los años que el empelado lleva en la compañía y parece ser un buen dato.
#    Dicha cantidad se puede calcular con la fecha de contratación ‘HiringDate’.
# 5. Crea una columna llamada Year y obtén el año de contratación del empleado a partir de su fecha ‘HiringDate’. No se te olvide que debe ser un entero.
EmpleadosAttrition['Year'] = EmpleadosAttrition['HiringDate'].str.split(pat='/').str[2].astype(int)

# 6. Crea una columna llamada YearsAtCompany que contenga los años que el empleado lleva en la compañía hasta el año 2018.
#    Para su cálculo, usa la variable Year que acabas de crear.
#EmpleadosAttrition['YearsAtCompany'] = datetime.now().year - EmpleadosAttrition['Year']  # => Esto sería para calcular los años al día de hoy
EmpleadosAttrition['YearsAtCompany'] = 2018 - EmpleadosAttrition['Year']

# 7. La DistanceFromHome está dada en kilómetros, pero tiene las letras “km” al final y así no puede ser entera.
# 8. Renombra la variable DistanceFromHome a DistanceFromHome_km
EmpleadosAttrition = EmpleadosAttrition.rename(columns = {'DistanceFromHome': 'DistanceFromHome_km'})

# 9. Crea una nueva variable DistanceFromHome que sea entera, es decir, solo con números
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome_km'].str.split().str[0].astype(int)

# 10. Borra las columnas Year, HiringDate y DistanceFromHome_km debido a que ya no son útiles.
EmpleadosAttrition = EmpleadosAttrition.drop(['Year', 'HiringDate', 'DistanceFromHome_km'],axis=1)

#11. Aprovechando los ajustes que se están haciendo, la empresa desea saber si todos los departamentos tienen un ingreso promedio similar.
#    Genera una nuevo frame llamado SueldoPromedioDepto que contenga el MonthlyIncome promedio por departamento de los empleados
#    y colócalo en una variable llamada SueldoPromedio. Esta tabla solo es informativa, no la vas a utilizar en el set de datos que estás construyendo.
SueldoPromedioDepto = EmpleadosAttrition.groupby(['Department'])[['MonthlyIncome']].mean().rename(columns = {'MonthlyIncome': 'SueldoPromedio'})
SueldoPromedioDepto

# 12. La variable MonthlyIncome tiene un valor numérico muy grande comparada con las otras variables.
#     Escala dicha variable para que tenga un valor entre 0 y 1.
EmpleadosAttrition['MonthlyIncome'] =  (EmpleadosAttrition['MonthlyIncome'] - min(EmpleadosAttrition['MonthlyIncome'])) / (max(EmpleadosAttrition['MonthlyIncome'])-min(EmpleadosAttrition['MonthlyIncome']))

# 13. Todo parece indicar que las variables categóricas que quedan sí son importantes para obtener la variable de salida.
#     Convierte todas las variables categóricas que quedan a numéricas:
# a) BusinessTravel
# b) Department
# c) EducationField
# d) Gender
# e) JobRole
# f) MaritalStatus
# g) Attrition


EmpleadosAttrition['BusinessTravel_Code'] = EmpleadosAttrition['BusinessTravel'].astype('category').cat.codes
EmpleadosAttrition['Department_Code'] = EmpleadosAttrition['Department'].astype('category').cat.codes
EmpleadosAttrition['EducationField_Code'] = EmpleadosAttrition['EducationField'].astype('category').cat.codes
EmpleadosAttrition['Gender_Code'] = EmpleadosAttrition['Gender'].astype('category').cat.codes
EmpleadosAttrition['JobRole_Code'] = EmpleadosAttrition['JobRole'].astype('category').cat.codes
EmpleadosAttrition['MaritalStatus_Code'] = EmpleadosAttrition['MaritalStatus'].astype('category').cat.codes
EmpleadosAttrition['Attrition_Code'] = EmpleadosAttrition['Attrition'].astype('category').cat.codes

# 14. Ahora debes hacer la evaluación de las variables para quedarte con las mejores.
#     Calcula la correlación lineal de cada una de las variables con respecto al Attrition
dfCorr = EmpleadosAttrition.drop(['BusinessTravel', 'Department', 'EducationField','Gender','JobRole','MaritalStatus','OverTime','Attrition'],axis=1)
# Con esto obtengo la matriz de correlaciones
dfCorr.corr()

# 15. Selecciona sólo aquellas variables que tengan una correlación mayor o igual a 0.1, dejándolas en otro frame llamado EmpleadosAttritionFinal.
#     No olvides mantener la variable de salida Attrition; esto es equivalente a borrar las que no cumplen con el límite.
EmpleadosAttritionFinal = EmpleadosAttrition[['Age','EnvironmentSatisfaction','JobInvolvement','JobLevel','JobSatisfaction','MonthlyIncome'
                                             ,'TotalWorkingYears','YearsInCurrentRole','YearsAtCompany','MaritalStatus_Code','Attrition_Code']]

# 16. Crea una nueva variable llamada EmpleadosAttritionPCA formada por los componentes principales del frame EmpleadosAttritionFinal.
#     Recuerda que el resultado del proceso PCA es un numpy array, por lo que, para hacer referencia a una columna,
#     por ejemplo, la 0, puedes usar la instrucción EmpleadosAttritionPCA[:,0]).

pca = PCA()
pca.fit(EmpleadosAttritionFinal)
EmpleadosAttritionPCA = pca.components_

EmpleadosAttritionPCA[:,0]

# 17. Agrega el mínimo número de Componentes Principales en columnas del frame EmpleadosAttritionPCA que logren explicar el 80% de la varianza,
# al frame EmpleadosAttritionFinal. Puedes usar la instrucción assign, columna por columna, llamando a cada unaC0, C1, etc., hasta las que vayas a agregar.
print(pca.explained_variance_)

# Resultados obtenidos:
# Age                = 1.22290764e+02  > 80% Sí explica la varianza
# EnvironmentSatisfaction = 4.65632154e+01  < 80% No logra explicar la varianza
# JobInvolvement     = 1.52078559e+01  < 80% No logra explicar la varianza
# JobLevel           = 4.05153475e+00  < 80% No logra explicar la varianza
# JobSatisfaction    = 1.25403224e+00  < 80% No logra explicar la varianza
# MonthlyIncome      = 1.24105572e+00  < 80% No logra explicar la varianza
# TotalWorkingYears  = 5.59071454e-01  < 80% No logra explicar la varianza
# YearsInCurrentRole = 5.48644069e-01  < 80% No logra explicar la varianza
# YearsAtCompany     = 5.36573142e-01  < 80% No logra explicar la varianza
# MaritalStatus_Code = 1.11649551e-01  < 80% No logra explicar la varianza

TransformedEmpleadosPCA = pca.transform(EmpleadosAttritionFinal)

EmpleadosAttritionFinal = EmpleadosAttritionFinal.assign(AgeC0 = TransformedEmpleadosPCA[:,0])
EmpleadosAttritionFinal

# 19. Guarda el set de datos que has formado y que tienes en EmpleadosAttritionFinal en un archivo CSV llamado EmpleadosAttritionFinal.csv.
# Las últimas columnas que colocaste quedarán después de la variable Attrition, lo cual no importa, pero si gustas lo puedes arreglar antes de escribir el archivo.
EmpleadosAttritionFinal = EmpleadosAttritionFinal[['Age','AgeC0','EnvironmentSatisfaction','JobInvolvement','JobLevel','JobSatisfaction','MonthlyIncome'
                                                  ,'TotalWorkingYears','YearsInCurrentRole','YearsAtCompany','MaritalStatus_Code','Attrition_Code']]
EmpleadosAttritionFinal.to_csv("/content/drive/MyDrive/M15/Reto/EmpleadosAttritionFinal.csv", index = False)


Mounted at /content/drive
[1.22290764e+02 4.65632154e+01 1.52078559e+01 4.05153475e+00
 1.25403224e+00 1.24105572e+00 5.59071454e-01 5.48644069e-01
 5.36573142e-01 1.11649551e-01 5.87988634e-03]
